In [22]:
logger="win10Davids"
import sys
sys.path.append('C:/Users/{}/Documents/AplicaTesis/2preprocesamiento'.format(logger))
from preprocesamiento import SQ_LENGTH,DICTIO_PATH
print(SQ_LENGTH)
archivo_salida='C:/Users/{}/Documents/AplicaTesis/4lstmcompas/analisis/'.format(logger)

64


In [23]:
import json
import numpy as np
import tensorflow.keras as keras
import music21 as m21


In [24]:
#SQ_LENGTH=38

In [25]:
#ROUTE_MODEL= "C:/Users/{}/Documents/AplicaTesis/3MODELO/cuecapura.h5".format(logger)
ROUTE_MODEL= "C:/Users/{}/Documents/AplicaTesis/3MODELO/cueca.h5".format(logger)
inisymbols = ["/"] * SQ_LENGTH

In [26]:

def model_load(model_path):
    """Load the pre-trained LSTM model."""
    return keras.models.load_model(model_path)



In [27]:
def dictio_load(mapping_path):
    """Load the mappings from the JSON file."""
    with open(mapping_path, "r") as fp:
        return json.load(fp)


In [28]:
def wide_variability(probabilities, temperature):
    """Sample an index from a probability array reapplying softmax using temperature."""
    predictions = np.log(probabilities) / temperature
    probabilities = np.exp(predictions) / np.sum(np.exp(predictions))

    choices = range(len(probabilities))
    index = np.random.choice(choices, p=probabilities)

    return index

In [29]:
def generate_compasmelody(model, mappings, seed, num_steps, max_sequence_length, temperature,nombre):
    """Generates a melody using the DL model and returns a list with symbols representing the melody."""
    seed = seed.split()
    melody = seed
    #archivo_salida='C:/Users/Davids/Documents/AplicaTesis/4lstmcompas/analisis/compuesto.txt'
    seed = inisymbols + seed

    seed = [mappings[symbol] for symbol in seed]

    for _ in range(num_steps):
        seed = seed[-max_sequence_length:]

        onehot_seed = keras.utils.to_categorical(seed, num_classes=len(mappings))
        onehot_seed = onehot_seed[np.newaxis, ...]

        probabilities = model.predict(onehot_seed)[0]
        output_int = variabilidad_amplia(probabilities, temperature)

        seed.append(output_int)

        output_symbol = [k for k, v in mappings.items() if v == output_int][0]
        
        #output_string = ', '.join(output_symbol)
        if output_symbol == "/":
            break

        melody.append(output_symbol)
       # output_string = ','.join(melody)
       # melodia_str = ' '.join(melody)
    
    # Guardar el contenido en el archivo de texto
    #with open(archivo_salida, 'w') as txt_file:
     #   txt_file.write(output_string) 
    #with open(nombre, 'w') as txt_file:
        #txt_file.write(melodia_str)   

    return melody

In [30]:
 def save_finmelody(melody, step_duration=0.125, format="midi", file_name="modeloaplicado2.mid"):
                # create a music21 stream
        stream = m21.stream.Stream()
        # Add the 6/8 time signature at the beginning of the stream
        time_signature = m21.meter.TimeSignature('6/8')
        stream.append(time_signature)
        start_symbol = None
        step_counter = 1

        # parse all the symbols in the melody and create note/rest objects
        for i, symbol in enumerate(melody):

            # handle case in which we have a note/rest
            if symbol != "_" or i + 1 == len(melody):

                # ensure we're dealing with note/rest beyond the first one
                if start_symbol is not None:

                    quarter_length_duration = ((step_duration * step_counter)/2) # 0.25 * 4 = 1

                    # handle rest
                    if start_symbol == "r":
                        m21_event = m21.note.Rest(quarterLength=quarter_length_duration)

                    # handle note
                    else:
                        m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)

                    stream.append(m21_event)

                    # reset the step counter
                    step_counter = 1

                start_symbol = symbol

            # handle case in which we have a prolongation sign "_"
            else:
                step_counter += 1
        stream.show
        # write the m21 stream to a midi file
        stream.write(format, file_name)
        

In [31]:
def save_tema(tema, nombre_archivo):
    # Convertir la lista a una cadena de texto
    tema_str = ' '.join(map(str, tema))

    # Guardar la cadena de texto en un archivo txt
    with open(nombre_archivo, "w") as file:
        file.write(tema_str)

In [32]:

def main():
    parte1="parte1"
    parte2="parte2"
    modelo = model_load(ROUTE_MODEL)    
    mapeo = dictio_load(DICTIO_PATH)
    #semilla1 = "60 _ _ _ 64 _ _ _ 67 _ _ _ 64 _ _ _ 59 _ _ _ 64 _ _ _"
    semilla1 = "67 _ _ _ _ _ _ _ _ _ _ _ 65 _ _ _ 64 _ _ _ _ _ _ _ 61 _ _ _ _ _ _ _ 57 _ _ _ _ _ _ _ 66 _ _ "
    #semilla2= "60 _ _ _ _ _62 _ _ _ _ _ _ _ _ _ 60 _ _ _ _68 _ _ _ 62 _ _ _ _ _ _ _"
    #"74 _ _ _ _ _ _ _ 71 _ _ _ _ _ _ _ 65 _ _ _ 61 _ _ _" 
    compas1 = generate_compasmelody(modelo, mapeo, semilla1, 500, SQ_LENGTH, 0.3,parte1)    
    #compas2 = generar_compasmelody(modelo, mapeo, semilla1, 500, SQ_LENGTH, 0.3,parte2)    
    #tema=compas1 +compas2 
    #guarda_finmelody(tema)
    save_finmelody(compas1)
    save_tema(compas1, archivo_salida + "compuesto2.txt")
    #guardar_tema(tema, archivo_salida + "compuesto1.txt")
    
    print("----END---")
   


In [33]:
if __name__ == "__main__":
    main()

1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 34ms/step
----END---
